## Scikit-Learn 모델 생성 방법

In [1]:
import findspark
findspark.init()

In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [3]:
#iris datasets 로딩
iris = load_iris()

iris_data  = iris.data # feature
iris_label = iris.target # label

iris_columns = ["sepal_length","sepal_width","petal_length","petal_width"]
iris_pdf=pd.DataFrame(iris_data, columns=iris_columns)
iris_pdf['target'] = iris_label
iris_pdf.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
# 훈련세트 데이터세트 분할 및 모델 생성
from sklearn.tree import DecisionTreeClassifier # Estimator
from sklearn.model_selection import train_test_split # RandomSpliter : 데이터를 무작위로 섞고 일정비율에 따라 잘라준다.

#데이터 무작위 섞고 비율로 나눠준다.
x_train, x_test, t_train, t_test = train_test_split(
    iris_data,
    iris_label,
    test_size=0.2,
    random_state=42
)

# 모델 생성
tree_clf = DecisionTreeClassifier()
# 훈련! log_reg 모델 자체에서 훈련이 일어나게 된다.
tree_clf.fit(x_train,t_train) 

pred = tree_clf.predict(x_test)
print(pred)


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


## ``Spark ML 사용해 보기``

In [5]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.master('local').appName('tree-classifier').getOrCreate()

In [6]:
# pandas 데이터 프레임을 spark 데이터 프레임으로 -> createDataFrame
iris_sdf = spark.createDataFrame(iris_pdf)
iris_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         5.1|        3.5|         1.4|        0.2|     0|
|         4.9|        3.0|         1.4|        0.2|     0|
|         4.7|        3.2|         1.3|        0.2|     0|
|         4.6|        3.1|         1.5|        0.2|     0|
|         5.0|        3.6|         1.4|        0.2|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



In [7]:
# spark DataFrame 에서 훈련, 테스트 데이터 분할 - randomSplit 메소드 활용
train_sdf, test_sdf = iris_sdf.randomSplit([0.8,0.2],seed=42) # train_data 비율 80% (test_data 비율 20%)

- 훈련 데이터 세트는 어떻게 변환이 되어도 하나만 존재하는 게 좋다.
- 훈련 데이터가 모델에 들어가면 모델이 transform을 해서 예측값이 나온다.
- 여러번 훈련 하면 transform 여러 번 일어난다.
- train_sdf가 메모리 내에 여러 개가 생길 수도 있다. 
- 전처리 -> 훈련 -> 예측 -> 평가 -> 훈련 -> 예측 -> 평가 -> 훈련 -> 예측 -> 평가
=> RDD가 여러개 생긴다. => 캐시에 넣고 활용하는 것이 좋다.

 ``cache를 적용하지 않으면 훈련을 할 때마다 sdf가 여러개 생긴다. => 좋은 방법 아니다.``

In [8]:
# 훈련 직전에 사용할 데이터는 캐싱을 하는 것이 유리!
train_sdf.cache()
# => 머신러닝 데이터로 사용 할 준비가 될 데이터는 cahce에 넣고 활용하자!

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, target: bigint]

In [9]:
train_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         4.3|        3.0|         1.1|        0.1|     0|
|         4.4|        2.9|         1.4|        0.2|     0|
|         4.4|        3.2|         1.3|        0.2|     0|
|         4.5|        2.3|         1.3|        0.3|     0|
|         4.6|        3.1|         1.5|        0.2|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



#### VectorAssembler를 이용하여 모든 feature 컬럼들을 하나의 feature vector로 변환
#### 하나의 백터로 합쳐주는 역할[4.3,3.0,1.1,0.1]

#### train 세트 transform : ``train_feature_vector_sdf=vec_assembler.transform(train_sdf)``

In [10]:

from pyspark.ml.feature import VectorAssembler

# 합쳐질 컬럼을 작성
iris_columns = ["sepal_length","sepal_width","petal_length","petal_width"]

# VectorAssembler로 데이터 프레임에 있는 데이터를 하나의 백터로 합쳐주기
vec_assembler=VectorAssembler(inputCols=iris_columns,outputCol="features")

# vectorAssembler_transform 

train_feature_vector_sdf=vec_assembler.transform(train_sdf)
train_feature_vector_sdf.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.3|        3.0|         1.1|        0.1|     0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|     0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.2|         1.3|        0.2|     0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|     0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|     0|[4.6,3.1,1.5,0.2]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



#### 모델 생성 : ``dt = DecisionTreeClassifier(featuresCol="features", labelCol="target", maxDepth=5)``
#### 모델 학습 : ``dt_model = dt.fit(train_feature_vector_sdf)``

In [11]:
# Spark ML의 모델은 Estimator지만, 데이터를 변환시키는 Transformer이다.
# train 데이터를 받아서 예측 값(pred)으로 변환시키는 transform 과정이 일어난다.
from pyspark.ml.classification import DecisionTreeClassifier

# 모델 생성. 어떤 컬럼의 데이터를 이용해서 학습을 할지 결정을 지어 준다.
# 예측을 ~할거야
dt = DecisionTreeClassifier(featuresCol="features", labelCol="target", maxDepth=5) # 나무 깊이 5단계

# 모델 학습. fit() 메소드를 이용하여 학습을 수행하고, 그 결과를 ML 모델로 반환한다.
dt_model = dt.fit(train_feature_vector_sdf)

# scikitlearn은 fit을 하게되면 모델 자체에서 훈련이 수행
# spark 모델은 fit을 하게 되면 그 결과로만 모델이 나온다.
print(type(dt))
print(type(dt_model))

<class 'pyspark.ml.classification.DecisionTreeClassifier'>
<class 'pyspark.ml.classification.DecisionTreeClassificationModel'>


In [12]:
# 테스트 데이터를 이용해서 예측
test_sdf.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|target|
+------------+-----------+------------+-----------+------+
|         4.4|        3.0|         1.3|        0.2|     0|
|         4.6|        3.2|         1.4|        0.2|     0|
|         4.6|        3.6|         1.0|        0.2|     0|
|         4.8|        3.1|         1.6|        0.2|     0|
|         4.9|        3.1|         1.5|        0.1|     0|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



#### train 세트 transform : train_feature_vector_sdf=vec_assembler.transform(train_sdf)
#### test 세트 transform : **``test_feature_vector_sdf= vec_assembler.transform(test_sdf)``**

In [13]:
# 훈련 데이터에서 적용시켰던 Transformer를 테스트 세트에 그대로 적용시킨다.
test_feature_vector_sdf= vec_assembler.transform(test_sdf)
test_feature_vector_sdf.show(5)
# 훈련세트의 transform은 테스트세트에서도  그대로 사용한다.

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



#### 예측 : predictions=dt_model.transform(test_feature_vector_sdf)

In [14]:
# 예측
# ==> 테스트 데이터 + 변환된 feature vector로 구성된 DataFrame을 활용한다.
# feature vector 란? vector_assembler 에 의해서 합쳐진 형태의 벡터를 활용한다.
# 모델의 transform() 메소드를 활용한다.
predictions=dt_model.transform(test_feature_vector_sdf)
predictions.show(5)

+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
+------------+-----------+------------+-----------+------+------

In [15]:
# 데이터들이 계속 합쳐지는 느낌이다. ==> RDD transformation이다.
# rawPrediction/ probability / prediction 추가된다.
# 기존 데이터프레임을 수정시키고 있다.

- ``rawPrediction`` : 머시러닝 모델 알고리즘 별로 다를 수 있다.
    - 머신러닝 알고리즘에 의해서 계산된 값
    - 값에 대한 정확한 의미는 없다!!!
    - LogisticRegression 의 경우 예측 label 별로, 예측 수행 전 ``sigmoid`` 함수 적용 전 값 (sigmoid는 0~1 사이 숫자)
        * $\hat{y} = \sigma(WX + b)$
        * $WX + b$의 결과가 `rawPrediction`
* `probability` : 예측 label 별 예측 확률

- ex)0번 클래스 100프로 예측
* `prediction: 최종 예측 label 값`

#### 예측값 : ``accuracy=evaluator_accuracy.evaluate(predictions) ``

In [16]:
# 정확도 판단해 보기
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # 다중분류

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="target",predictionCol="prediction",metricName="accuracy")
# metricname 은 평가방법

accuracy=evaluator_accuracy.evaluate(predictions) # 예측값이 들어있는 predictions dataframe이 들어간다.
print("정확도",accuracy)


정확도 1.0


## LogisiticRegression 사용해 보기

In [17]:
from pyspark.ml.classification import LogisticRegression

# ML 알고리즘 객체 생성
lr =LogisticRegression(featuresCol='features',labelCol='target',maxIter=10)

lr_model= lr.fit(train_feature_vector_sdf)

predictions=lr_model.transform(test_feature_vector_sdf)
predictions.show(5)

accuracy=evaluator_accuracy.evaluate(predictions) # 예측값이 들어있는 predictions dataframe이 들어간다.
print("정확도",accuracy)

+------------+-----------+------------+-----------+------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------------+--------------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[18.6086266693526...|[0.99997762791224...|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[18.8180066107263...|[0.99997581287298...|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[22.6963845270307...|[0.99999942608846...|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[16.7506644665745...|[0.99971232954776...|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[17.3393987944

## 파이프라인 구축
- 만드는 이유 : `pipeline`은 여러 개의 개별적인 `transformer`의 변환 작업, `Estimator`의 학습 작업을 일련의 process 연결을 통해 간단한 API 처리로 구현할 수 있게 만들어 준다.

#### 파이프라인은 개별 변환 및 모델 학습 작업을 stage로 각각 정의한다.
- 정의하고 파이프라인에 등록한다.
- Pipeline.fit() 메소드를 활용해서 순서대로 연결된 stage 작업을 일괄적으로 수행한다.
- Pipeline.fit()의 결과물은 pipelineModel이 반환된다.
- PipelineModel에서 예측 작업을 transform()으로 수행한다.

#### 정리
1. stage 정의
2. 연속적으로 이어준 stage들을 파이프라인에 등록   
3. 파이프라인에서 fit()하면 모델이 등장
4. 모델에서 tramsform을 통해서 prediction 작업 수행

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

iris_columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

# 첫번째 stage는 Feature Vectorization을 위해 VectorAssembler 객체 생성
stage_1 = VectorAssembler(inputCols=iris_columns, outputCol='features')

# 두번째 stage는 학습을 위한 모델 생성
stage_2 = DecisionTreeClassifier(featuresCol='features', labelCol='target',maxDepth=5)

# 리스트를 이용해 stage를 순서대로 묶어주기
stages=[stage_1,stage_2]

# 파이프라인에 등록
pipeline=Pipeline(stages=stages)

print(type(pipeline))


<class 'pyspark.ml.pipeline.Pipeline'>


In [19]:
# Transformation 작업을 수행할 데이터 프레임을 fit한다.
pipeline_model = pipeline.fit(train_sdf)
print(type(pipeline_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


In [20]:
predictions = pipeline_model.transform(test_sdf)
predictions.show(5)

+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|target|         features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+--------------+-------------+----------+
|         4.4|        3.0|         1.3|        0.2|     0|[4.4,3.0,1.3,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.2|         1.4|        0.2|     0|[4.6,3.2,1.4,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.6|         1.0|        0.2|     0|[4.6,3.6,1.0,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.8|        3.1|         1.6|        0.2|     0|[4.8,3.1,1.6,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.9|        3.1|         1.5|        0.1|     0|[4.9,3.1,1.5,0.1]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
+------------+-----------+------------+-----------+------+------

In [21]:
spark.stop()